In [188]:
import json
import pandas as pd
import numpy as np
from collections import Counter
import seaborn as sns
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import pos_tag
from string import punctuation,digits
import re
from keras.utils import to_categorical
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from pandas_ml import ConfusionMatrix
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers import Bidirectional
from keras.layers import Embedding
# from keras.utils.np_utils import probas_to_classes
from sklearn import preprocessing
from keras.models import Sequential
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score,accuracy_score
from sklearn.model_selection import train_test_split
%matplotlib inline
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from matplotlib import rcParams
from random import shuffle
# figure size in inches
rcParams['figure.figsize'] = 11.7,8.27
import word2vecReader as godin_embedding
from gensim.models import KeyedVectors

In [189]:
def load_data_from_json(filename):
    with open(filename,'r') as f:
        return json.load(f)

In [190]:
master_dict = load_data_from_json('dataset/master_train.json')

In [191]:
def extract_data_from_dict(master_dict):
    sentence = []
    aspect = []
    for key in master_dict.keys():
        for info in master_dict[key]['info']:
            sentence.append(eval(info['snippets'])[0])
            aspect.append(eval(info['aspects'])[0])
    return sentence,aspect

In [192]:
sentence,aspect = extract_data_from_dict(master_dict)

In [193]:
len(sentence),len(aspect)

(1171, 1171)

In [194]:
aspect_level1= []
aspect_level2=[]
for asp in aspect:
    try:
        aspect_level1.append(asp.split('/')[0])
        aspect_level2.append(asp.split('/')[1])
    except:
        print(asp)

In [195]:
def load_data_from_tsv_headline(filename):
    df = pd.read_csv(filename, sep='\t')
    df.columns = ["id", "sentence", "snippet", "score","aspect"]
    snippet = df['snippet'].tolist()
    aspect = df['aspect'].tolist()
    
    aspect_l2 = []
    for asp in aspect:
        try:
            aspect_l2.append(asp.split('/')[1])
        except:
            print(asp)
    
    return snippet,aspect_l2

In [196]:
test_headline_snippet,test_headline_aspect=load_data_from_tsv_headline('dataset/test_headlines_samples - Sheet1.tsv')

In [197]:
def load_data_from_tsv_post(filename):
    df = pd.read_csv(filename, sep='\t')
    df.columns = ["id", "sentence", "snippet", "score",'af','as1','as2','as3','as4']
    snippet = df['snippet'].tolist()
    aspect = df['as2'].tolist()
    return snippet,aspect

In [198]:
test_post_snippet,test_post_aspect = load_data_from_tsv_post('dataset/test_set_post - Sheet1.tsv')

In [199]:
def remove_punctuation(s):
    list_punctuation = list(punctuation.replace('$',''))
    for i in list_punctuation:
        s = s.replace(i,'')
    return s

In [200]:
def clean_sentence(sentence):
    #remove multiple repeat non num-aplha char !!!!!!!!!-->!
    sentence = re.sub(r'(\W)\1{2,}', r'\1', sentence) 
    #removes alpha char repeating more than twice aaaa->aa
    sentence = re.sub(r'(\w)\1{2,}', r'\1\1', sentence)
    #removes links
    sentence = re.sub(r'(?P<url>https?://[^\s]+)', r'', sentence)
    # remove @usernames
    sentence = re.sub(r"\@(\w+)", "", sentence)
    #remove # from #tags
    sentence = sentence.replace('#','')
    # split into tokens by white space
    tokens = sentence.split()
    # remove punctuation from each token
    tokens = [remove_punctuation(w) for w in tokens]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    tokens = ' '.join(tokens)
    return tokens

In [201]:
len(sentence),len(aspect_level2),len(test_post_snippet),len(test_post_aspect),len(test_headline_snippet),len(test_headline_aspect)

(1171, 1171, 99, 99, 93, 93)

In [202]:
dataX = [clean_sentence(x) for x in sentence]
test_headlineX = [clean_sentence(x) for x in test_headline_snippet]
test_postX = [clean_sentence(x) for x in test_post_snippet]

In [203]:
max_length  = 11

In [204]:
no_of_classes = 27

In [205]:
dataY = aspect_level2

In [206]:
def convert_lables(trainY):
    le = preprocessing.LabelEncoder()
    le.fit(trainY)
    temp1 = le.transform(trainY)
    return to_categorical(temp1,no_of_classes),le.classes_

In [207]:
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [208]:
# encode a list of lines
def encode_text(tokenizer, lines, length):
    encoded = tokenizer.texts_to_sequences(lines)
    padded = pad_sequences(encoded, maxlen=length, padding='post')
    return padded

In [209]:
#loading Google Word2Vec
def load_google_word2vec(file_name):
    return KeyedVectors.load_word2vec_format(file_name, binary=True)

In [210]:
def get_word2vec_embedding_matrix(model):
    embedding_matrix = np.zeros((vocab_size,300))
    for word, i in tokenizer.word_index.items():
        try:
            embedding_vector = model[word]
        except KeyError:
            embedding_vector = None
        if embedding_vector is not None:
            embedding_matrix[i]=embedding_vector
    return embedding_matrix

In [211]:
def define_model(learning_rate,dropout,lstm_out,n_hidden_layer,em,em_trainable_flag,free_em_dim):
    model = Sequential()
    
    if em == 'free':
        model.add(Embedding(vocab_size, free_em_dim))
    else:
        model.add(Embedding(vocab_size, len(eval(em)[0]), weights = [eval(em)],input_length=max_length,trainable = em_trainable_flag))
    model.add(Bidirectional(LSTM(lstm_out, recurrent_dropout=dropout)))
    for i in range(n_hidden_layer):
        model.add(Dense(int((2*lstm_out+no_of_classes)/2),activation='relu'))
    model.add(Dense(no_of_classes,activation='softmax'))
    optimizer = Adam(lr=learning_rate)
    model.compile(loss = 'categorical_crossentropy', optimizer=optimizer,metrics = ['accuracy'])
    print(model.summary())
    return model

In [212]:
word2vec_model= load_google_word2vec('/home/hitkul/Desktop/ps3_iiit/ensembl_work/ensemble_experiments/word_embeddings/GoogleNews-vectors-negative300.bin')

In [213]:
tokenizer = create_tokenizer(dataX)
vocab_size = len(tokenizer.word_index) + 1
trainX = encode_text(tokenizer, dataX, max_length)
test_headlineX = encode_text(tokenizer, test_headlineX, max_length)
test_postX = encode_text(tokenizer, test_postX, max_length)
trainY,lable_encoding = convert_lables(dataY)

In [214]:
embedding_matrix_word2vec = get_word2vec_embedding_matrix(word2vec_model)

In [215]:
model = define_model(learning_rate=0.0006296220146941253,
                     dropout=0.5,
                     lstm_out=400,
                     n_hidden_layer=1,
                     em='embedding_matrix_word2vec',
                     em_trainable_flag=False,
                     free_em_dim=400)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 11, 300)           633900    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 800)               2243200   
_________________________________________________________________
dense_1 (Dense)              (None, 413)               330813    
_________________________________________________________________
dense_2 (Dense)              (None, 27)                11178     
Total params: 3,219,091
Trainable params: 2,585,191
Non-trainable params: 633,900
_________________________________________________________________
None


In [216]:
history_object = model.fit(trainX, trainY, epochs=100,batch_size=8)

Epoch 1/100
1171/1171 [==============================] - 19s 16ms/step - loss: 2.0695 - acc: 0.4705
Epoch 2/100
1171/1171 [==============================] - 18s 15ms/step - loss: 1.2265 - acc: 0.6610
Epoch 3/100
1171/1171 [==============================] - 18s 15ms/step - loss: 0.9051 - acc: 0.7464
Epoch 4/100
1171/1171 [==============================] - 17s 15ms/step - loss: 0.6735 - acc: 0.7976
Epoch 5/100
1171/1171 [==============================] - 17s 15ms/step - loss: 0.5600 - acc: 0.8207
Epoch 6/100
1171/1171 [==============================] - 17s 15ms/step - loss: 0.4353 - acc: 0.8728
Epoch 7/100
1171/1171 [==============================] - 17s 15ms/step - loss: 0.3341 - acc: 0.8907
Epoch 8/100
1171/1171 [==============================] - 17s 15ms/step - loss: 0.2629 - acc: 0.9172
Epoch 9/100
1171/1171 [==============================] - 18s 15ms/step - loss: 0.2467 - acc: 0.9172
Epoch 10/100
1171/1171 [==============================] - 17s 15ms/step - loss: 0.2027 - acc: 0.9360

Epoch 82/100
1171/1171 [==============================] - 17s 15ms/step - loss: 0.0361 - acc: 0.9889
Epoch 83/100
1171/1171 [==============================] - 18s 16ms/step - loss: 0.0288 - acc: 0.9872
Epoch 84/100
1171/1171 [==============================] - 18s 15ms/step - loss: 0.0172 - acc: 0.9923
Epoch 85/100
1171/1171 [==============================] - 17s 15ms/step - loss: 0.0148 - acc: 0.9932
Epoch 86/100
1171/1171 [==============================] - 17s 15ms/step - loss: 0.0139 - acc: 0.9940
Epoch 87/100
1171/1171 [==============================] - 18s 15ms/step - loss: 0.0151 - acc: 0.9906
Epoch 88/100
1171/1171 [==============================] - 18s 15ms/step - loss: 0.0135 - acc: 0.9940
Epoch 89/100
1171/1171 [==============================] - 18s 15ms/step - loss: 0.0119 - acc: 0.9932
Epoch 90/100
1171/1171 [==============================] - 18s 15ms/step - loss: 0.0123 - acc: 0.9932
Epoch 91/100
1171/1171 [==============================] - 18s 15ms/step - loss: 0.0139 - ac

In [217]:
def get_class_from_pred(pred):
    return [lable_encoding[x.argmax()] for x in pred]

In [218]:
pred_headline = model.predict(test_headlineX)

In [219]:
pred_class_headline = get_class_from_pred(pred_headline)

In [220]:
f1_score(test_headline_aspect,pred_class_headline,average='macro')

/home/hitkul/anaconda3/envs/ps3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/hitkul/anaconda3/envs/ps3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.14448682630500811

In [221]:
f1_score(test_headline_aspect,pred_class_headline,average='weighted')

/home/hitkul/anaconda3/envs/ps3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/hitkul/anaconda3/envs/ps3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.32704463994786576

In [222]:
# 

In [223]:
pred_post = model.predict(test_postX)

In [224]:
pred_class_post = get_class_from_pred(pred_post)

In [225]:
f1_score(test_post_aspect,pred_class_post,average='macro')

/home/hitkul/anaconda3/envs/ps3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/hitkul/anaconda3/envs/ps3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.3501794477404233

In [226]:
f1_score(test_post_aspect,pred_class_post,average='weighted')

/home/hitkul/anaconda3/envs/ps3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/hitkul/anaconda3/envs/ps3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.8980628818027192